In [1]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 



%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')

global times
from timeit import default_timer as time

from matplotlib import rc

## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})
rc('text', usetex=True)

In [2]:
def cylinder(ri, ro, he = 1.0, element_order = 1, quads = False, 
           quad_incomplete = 1, quad_linear = 1, structured = False, save=False):
    """
    PORPUSE: Define a cylinder section mesh.
    """
    #***************************************************************************************************
    import gmsh

    gmsh.initialize()
    # gmsh.fltk.initialize()

    # Lets create a simple square element:
    model = gmsh.model
    geo   = model.geo
    option= gmsh.option

    P1 = geo.addPoint( 0.0, 0.0, 0.0, he, 1)
    P2 = geo.addPoint( 0.0,  ri, 0.0, he, 2)
    P3 = geo.addPoint(  ri, 0.0, 0.0, he, 3)
    P4 = geo.addPoint( 0.0,  ro, 0.0, he, 4)
    P5 = geo.addPoint(  ro, 0.0, 0.0, he, 5)

    C1 = geo.addCircleArc(P2,P1,P3)
    C2 = geo.addCircleArc(P5,P1,P4)
    
    geo.remove([P1])
    
    L1 = geo.addLine(P4,P2) # Left
    L2 = geo.addLine(P3,P5) # bottom

    loop1 = geo.addCurveLoop([C1,L2,C2,L1])

    surface = geo.addPlaneSurface([loop1])

    model.addPhysicalGroup( 0, [P1],        100 )
    model.addPhysicalGroup( 0, [P2],        101 )
    model.addPhysicalGroup( 0, [P3],        102 )
    model.addPhysicalGroup( 0, [P4],        103 )
    model.addPhysicalGroup( 0, [P5],        104 )

    model.addPhysicalGroup( 1, [C1],        201 )
    model.addPhysicalGroup( 1, [C2],        202 )
    model.addPhysicalGroup( 1, [L1],        203 )
    model.addPhysicalGroup( 1, [L2],        204 )

    model.addPhysicalGroup( 2, [surface],   301 )
    
    model.setPhysicalName( 0, 100, "center"  )
    model.setPhysicalName( 0, 101, "ri_left"  )
    model.setPhysicalName( 0, 102, "ri_right" )
    model.setPhysicalName( 0, 103, "ro_left"  )
    model.setPhysicalName( 0, 104, "ro_right" )

    model.setPhysicalName( 1, 201, "ri"    )
    model.setPhysicalName( 1, 202, "ro"    )
    model.setPhysicalName( 1, 203, "left"  )
    model.setPhysicalName( 1, 204, "right" )

    model.setPhysicalName( 2, 301, "domain"      )

    
    geo.synchronize()

    option.setNumber('Mesh.ElementOrder',element_order)


    # Recombine if quads are wanted.
    if quads:

        # Set body to recombine into quads.
        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         1)

    else:

        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         0)

    # end if 

    if structured:
        nels_L = int((ro-ri)/he)+1
        nels_C = min(int((ri*np.pi*2)/(4*he))+1,100)
        model.mesh.setTransfiniteCurve(L1,nels_L)
        model.mesh.setTransfiniteCurve(L2,nels_L)
        model.mesh.setTransfiniteCurve(C1,nels_C)
        model.mesh.setTransfiniteCurve(C2,nels_C)
        model.mesh.setTransfiniteSurface(surface)#,"left",[P2,P3,P4])
    # end if 

    model.mesh.generate(2)

    if save:
        gmsh.write("cylinder.msh")
    # end if 

    Th = fem.mesh.from_gmsh(gmsh)
#     Th = None

    gmsh.finalize()

    return Th

#-----------------------------------------------------------------------------------------------------

In [10]:
fem.set_global_algebra(oti)
alg = fem.get_global_algebra()

Th  = cylinder(1,2,he=0.01,element_order=1,quads=True,save=False, structured=True)
Th2 = cylinder(1,2,he=0.01,element_order=1,quads=True,save=False, structured=True)
Th

< mesh (pyoti.sparse) object with 10101 nodes, 10303 elements of types ( point1 (5), line2 (398), quad4 (9900) ) >

In [37]:
order = 1
ro = alg.number(2) + alg.e(1, order = order)
ri = alg.number(1)
# Perturb nodal coordinates for ro
pc = 0.1
for i in range(Th.x.size):
    
    x = Th.x[i,0]
    y = Th.y[i,0]
    
    r = alg.sqrt(x**2+y**2)
    if r.real > (ro-pc*(ro-ri)).real:
        
        h = (1-(ro-r)/(pc*(ro-ri))).real

        hx = h*x.real/r.real
        hy = h*y.real/r.real

        Th.x[i,0] = x.real + hx*alg.e(1, order = order)
        Th.y[i,0] = y.real + hy*alg.e(1, order = order)
    else:
        Th.x[i,0] = x.real
        Th.y[i,0] = y.real
# end for 

In [38]:
import pyvista as pv
p = pv.BackgroundPlotter()

In [39]:

Th2.x = Th.x.taylor_integrate([1],[1.0])
Th2.y = Th.y.taylor_integrate([1],[1.0])
grid = Th2.to_pv(dims=2,pd=[np.sqrt(Th.x.get_deriv([1])**2+Th.y.get_deriv([1])**2).real],pd_names=['pert'])

p.clear()

p.add_mesh(grid, show_edges=True, line_width=2,cmap='jet')


p.show_bounds()
p.show_grid()

(vtkRenderingAnnotationPython.vtkCubeAxesActor)0x7fa56969f600

In [35]:
Th.x.get_deriv([1])**2

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.77420506],
       [0.84618694],
       [0.92136801]])